In [1]:
import numpy as np
import pandas as pd
import random
import time

from sklearn.cluster import KMeans
from sklearn.datasets import load_iris
from sklearn.metrics import silhouette_score

random.seed(1234)

In [2]:
class experimento:
    # pontos é um np array com pontos nas linhas
    # resposta é um np array com as labels dos pontos
    def __init__(self, K, pontos, labels, p=1):
        # Parâmetro da distancia de Minkowski
        self.p = p
        self.N = pontos.shape[0]
        self.K = K
        
        self.pontos = pontos
        self.labels = labels
        
        # Calcula a matriz de distância usando norma p 
        t_inicial = time.time()
        self._calcular_distancias()
        t_final = time.time()
        print("Tempo para calcular matrix de dist: ", t_final - t_inicial)
        
    def _calcular_distancias(self):
        """
        # Calculando dist matrix
        self.dist_matrix = np.zeros((self.N, self.N))
        for i in range(self.N):
            for j in range(self.N):
                self.dist_matrix[i][j] = np.sum(np.abs(self.pontos[i] - self.pontos[j]) ** self.p) ** (1 / self.p)
                
        # Salvando na memória
        np.save('test_dist.npy', self.dist_matrix)
        """
        # Carregando pre computado da memória
        self.dist_matrix = np.load('test_dist.npy')
        self.dist_matrix = np.load(mem_file)
                
    # Constrói uma solução que possui raio, no máximo 2*r, ou retorna impossível
    def aprox_raio(self, r):
        centros = []
        ok = [False]*self.N
        idx = list(range(self.N))
        random.shuffle(idx)
        for i in range(self.N):
            if ok[i]:
                continue
            centros.append(idx[i])
            for j in range(i+1, self.N):
                if ok[j]:
                    continue
                if self.dist_matrix[idx[i]][idx[j]] <= 2*r:
                    ok[j] = True

        if len(centros) <= K:
            return centros
        else:
            return []

    # Retorna solução aproximada no máximo 2*ótimo com busca binária
    def aprox_refinamento(self, ):
        lo = 0
        hi = 0
        for i in range(len(pontos)):
            for j in range(i+1, len(pontos)):
                hi = max(hi, self.dist_matrix[i][j])

        EPS = 1e-12
        while hi - lo > EPS:
            mid = (lo+hi)/2
            if len(self.aprox_raio(mid)) > 0:
                lo = mid
            else:
                hi = mid

        return self.aprox_raio((lo+hi)/2)

    # Retorna solução aproximada no máximo 2*ótimo com método incremental
    def aprox_incremental(self):
        if K >= self.N:
            return list(range(self.N))
        
        start = random.randint(0, self.N-1)
        centros = [ start ]
        
        falta = list(range(self.N))
        falta.remove(start)
        while len(centros) < self.K:
            idx = -1 
            mxdist_matrix = -1
            for i in falta:
                idist_matrix = 0
                for c in centros:
                    idist_matrix = max(idist_matrix, self.dist_matrix[i][c])
                if idist_matrix > mxdist_matrix:
                    idx = i
                    mxdist_matrix = idist_matrix

            centros.append(idx)
            falta.remove(idx)

        return centros
    
    def kmeans(self):
        kmeans = KMeans(n_clusters=self.K, n_init=10, random_state=random.randint(0, 1000))
        labels = kmeans.fit_predict(self.pontos)
        centros = kmeans.cluster_centers_
        return centros
    
    def particao(self, centros):
        labels = [-1]*self.N
        for i in range(self.N):
            mn = -1
            for c in centros:
                if self.dist_matrix[i][c] < self.dist_matrix[i][mn]:
                    mn = c
            labels[i] = c
        return labels

    def teste_incremental(self):
        NUM_IT = 30
        for i in range(NUM_IT):
            t_inicial = time.time()
            centros = self.aprox_incremental()
            t_final = time.time()
            
            print(f"Tempo para teste {i} = {t_final - t_inicial}")
            
            labels = self.particao(centros)
            # silhueta = silhouette_score(self.pontos, labels)
            
            #rand = adjusted_rand_score(self.resposta, labels)

In [3]:
import dados

K, pontos, labels = dados.banknote_data()
E = experimento(K, pontos, labels, 1)

NameError: name 'mem_file' is not defined

In [ ]:
E.teste_incremental()

Tempo para teste 0 = 0.0014214515686035156
Tempo para teste 1 = 0.0007798671722412109
Tempo para teste 2 = 0.0008566379547119141
Tempo para teste 3 = 0.0007860660552978516
Tempo para teste 4 = 0.0016632080078125
Tempo para teste 5 = 0.0008518695831298828
Tempo para teste 6 = 0.0006794929504394531
Tempo para teste 7 = 0.00070953369140625
Tempo para teste 8 = 0.0009098052978515625
Tempo para teste 9 = 0.0007522106170654297
Tempo para teste 10 = 0.0007445812225341797
Tempo para teste 11 = 0.0014069080352783203
Tempo para teste 12 = 0.001499176025390625
Tempo para teste 13 = 0.0007574558258056641
Tempo para teste 14 = 0.0007486343383789062
Tempo para teste 15 = 0.0007367134094238281
Tempo para teste 16 = 0.0007083415985107422
Tempo para teste 17 = 0.0008673667907714844
Tempo para teste 18 = 0.0007364749908447266
Tempo para teste 19 = 0.0006909370422363281
Tempo para teste 20 = 0.0007004737854003906
Tempo para teste 21 = 0.0007343292236328125
Tempo para teste 22 = 0.0007116794586181641
Temp